In [47]:
import numpy as np
import pandas as pd

import os
#os.sys.path
import sys
sys.path.append('../src')

from pathlib import Path

import matplotlib.pyplot as plt
from PIL import Image
import cv2

In [69]:
# importing tensorflow model libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import model_from_json,load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import *
import tensorflow.keras.backend as K
import json
import time

### IMGS

In [70]:
inp1 = Path.home()/'Iron'/'inp1'

In [77]:
train_imgs = inp1/'TRAIN'
train_imgs_haar = inp1/'TRAIN_haar'
val_imgs = inp1/'VALIDATION'
test_imgs = inp1/'TEST'
demo_imgs = Path.cwd().parent/'demo'
demo_imgs_faces = Path.cwd().parent/'demo_haar'
demo_imgs_haar = Path.cwd().parent/'demo_faces'

### DATASETS to check

In [78]:
df_fer = pd.read_csv(inp1/'Fer.csv',encoding = "ISO-8859-1")
df_train = pd.read_csv(inp1/'Training_Data.csv',encoding = "ISO-8859-1")
df_test = pd.read_csv(inp1/'Testing_Data.csv',encoding = "ISO-8859-1")
df_val = pd.read_csv(inp1/'Validation_Data.csv',encoding = "ISO-8859-1")

In [79]:
face_cascade = cv2.CascadeClassifier('../src/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('../src/haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier('../src/haarcascade_smile.xml')

In [80]:
x = pd.DataFrame

In [82]:
def transfImag(path, new_path):
    """
    recibe carpeta, en cada foto de esa carpeta:
    lectura
    to gray
    facecascade
    por xywh en cada cara:
        array
        reshape array a 2d
        stack array 3d
        normalizar
        expand 4d
    devuelve array x cada foto para pasarselo al modelo
    """
    counter_imgs = 0
    counter_faces = 0
    
    for file in sorted(path.iterdir()):
        
        counter_imgs += 1

        input_img1 = cv2.imread(str(file))
        input_img2 = cv2.cvtColor(input_img1, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(input_img2, 1.25, 6)
        
        for (x,y,w,h) in faces:
            
            counter_faces += 1
            img_data1 = input_img2 [y:y+h,x:x+w]
            img_data2 = cv2.resize (img_data1,(48,48))

            img_data3 = np.stack(img_data2) 
            img_data4 = img_data2 / 255.0
            img_data5 = img_data3 / 255.0
            
            img_data6 = np.expand_dims(img_data5,axis=0).reshape(np.expand_dims(img_data5,axis=0).shape[0], 48, 48, 1)
            
            print(img_data6.shape)
            
            img_datashow = img_data3*255
            
            img_show = Image.fromarray(img_datashow)
            
            file_to_save = file.name.replace(".",f"_face{counter_faces}.")
            
            img_show.save(str(new_path/file_to_save))
            
            counter_faces = 0
            
            arr_for_model = img_data6
            
            
            return arr_for_model
    
    print('YAAA!')    
    return counter_imgs
        
    

In [83]:
transfImag(demo_imgs, demo_imgs_faces)

(1, 48, 48, 1)


array([[[[0.26666667],
         [0.41568627],
         [0.29803922],
         ...,
         [0.36862745],
         [0.37254902],
         [0.36078431]],

        [[0.2745098 ],
         [0.23137255],
         [0.25098039],
         ...,
         [0.35294118],
         [0.33333333],
         [0.34117647]],

        [[0.2745098 ],
         [0.28627451],
         [0.1372549 ],
         ...,
         [0.35294118],
         [0.34901961],
         [0.37254902]],

        ...,

        [[0.00392157],
         [0.05882353],
         [0.10980392],
         ...,
         [0.22745098],
         [0.25490196],
         [0.12156863]],

        [[0.00392157],
         [0.06666667],
         [0.13333333],
         ...,
         [0.28627451],
         [0.11372549],
         [0.06666667]],

        [[0.00392157],
         [0.06666667],
         [0.14509804],
         ...,
         [0.25490196],
         [0.16862745],
         [0.15686275]]]])

In [ ]:
def base_model():
    model = Sequential()
    input_shape = (48,48,1)
    #1st convolution layer
    model.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    #2nd convolution layer
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    #3rd convolution layer
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(7))
    model.add(Activation('softmax'))

    my_optimiser = tf.keras.optimizers.Adam(
    learning_rate = 0.001, beta_1=0.9, beta_2=0.999, 
        epsilon=1e-07, amsgrad=False, name='Adam')
    
    model.compile(loss='categorical_crossentropy',
                  metrics=['accuracy'],
                  optimizer=my_optimiser)
    
    return model

In [ ]:
model_1 = base_model()

model_v1 = model_1.fit(X_train, y_train, 
            validation_data=(X_val, y_val), 
            epochs=15,
            verbose=1, 
            batch_size=50)

model_1.summary()

In [7]:
# HAAR CASCADE CLASSIFIER
def detect_face_eyes_smile(pth,new_pth):
    """
    Extracts all .jpg files from local path, 
    calls on haar cascade classifiers (frontalface, eyes and smile) 
    and draws detection rectangles on each .jpg 
    
    Takes: local path of directory with .jpg images
    
    Returns: individual windows for .jpg files with detection rectangles for face, eyes and smile
    """

    counter_imgs = 0
    counter_faces = 0
    counter_smiles = 0
    counter_eyes = 0
    face_cascade = cv2.CascadeClassifier('../src/haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier('../src/haarcascade_eye.xml')
    smile_cascade = cv2.CascadeClassifier('../src/haarcascade_smile.xml')
    
    for file in sorted(pth.iterdir()):
        if file.suffix != '.jpg':
            pass
        else:
            counter_imgs += 1
            print(file.name)
            
            img = cv2.imread(str(file))
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            #plt.imshow(img)

            # FRONTAL FACE 
            
            faces = face_cascade.detectMultiScale(
                img_gray, 
                scaleFactor=1.06,                
                minNeighbors=7,
                minSize=(30, 30), 
                flags=cv2.CASCADE_SCALE_IMAGE)
            if faces is None:
                print("No Face Found")

            for (fx,fy,fw,fh) in faces:
                
                counter_faces += 1
                roi_gray = img_gray [fy:fy+fh, fx:fx+fw] # region of interest for detection
                roi_gray2 = cv2.resize (roi_gray, (48,48))
                roi_gray3 = np.stack(roi_gray2)
                roi_gray4 = roi_gray2 / 255
                
                roi_color = img[fy:fy+fh, fx:fx+fw] # region of interest for mapping rectangle
                roi_color2 = cv2.resize (roi_color, (48,48))
                roi_color3 = np.stack(roi_color2)
                roi_color4 = roi_color2 / 255
                
                cv2.rectangle(
                    img,
                    (fx,fy),
                    (fx+fw,fy+fh),
                    #(127,0,255),
                    (0,255,0),
                    2)

                # SMILES 

                smiles = smile_cascade.detectMultiScale(
                    roi_gray, 
                    scaleFactor = 1.35, 
                    minNeighbors = 8)

                for (sx, sy, sw, sh) in smiles:
                    counter_smiles += 1
                    cv2.rectangle(
                        roi_color,
                        (sx, sy),
                        (sx + sw, sy + sh),
                        #(255, 0, 130),
                        #(0,220,80),
                        (127,0,255),
                        1)

                # EYES

                eyes = eye_cascade.detectMultiScale(
                    roi_gray,
                    scaleFactor=1.05,
                    minNeighbors = 6)

                for (ex,ey,ew,eh) in eyes:
                    counter_eyes += 1
                    cv2.rectangle(
                        roi_color, 
                        (ex , ey),
                        (ex + ew, ey + eh),
                        (0,255,255),
                        1)
            
                # save images with detected regions
                file_to_save = file.name.replace(".",f"_face{counter_faces}.")
                
                #cv2.imwrite(str(pth.parent/'demo_faces'/file_to_save),img)
                cv2.imwrite(str(new_pth/file_to_save),roi_color)
                counter_imgs = 0
                counter_faces = 0
            # show the output frame
            cv2.imshow(f"img{file_to_save}", img)
            key = cv2.waitKey(100) & 0xFF
            
    cv2.destroyAllWindows(f"img{file_to_save}")
    

            
            

    
    """
        # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                # do a bit of cleanup
                cv2.destroyAllWindows()
                break
        
    # do a bit of cleanup
    cv2.destroyAllWindows()
cv2.destroyAllWindows()


        """



In [8]:
demo_imgs = Path.cwd().parent/'demo'
demo_imgs_faces = Path.cwd().parent/demo2
demo1 = demo_imgs/'A_0.jpg'
demo2 = demo_imgs/'demooo_01.jpg'

In [9]:
face_cascade = cv2.CascadeClassifier('../src/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('../src/haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier('../src/haarcascade_smile.xml')

In [ ]:
cv2.imwrite(str(pth.parent/*'_haar'/file.name),roi_color)

In [ ]:
str(pth.parent/'*_haar'/file.name)